# Extract workspace details from portal URI

In [101]:
# Ask the user to paste in the URL from the portal
portal_ws_uri = (
    "https://ms.portal.azure.com/#@microsoft.onmicrosoft.com/resource/subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourceGroups/soc/providers/Microsoft.OperationalInsights/workspaces/cybersecuritysoc/Overview"
)

import re

resid_pattern = (
    r"https://.*@(?P<tenantname>[^/]+)/resource(?P<res_id>/subscriptions/.*)"
)
ws_match = re.search(resid_pattern, portal_ws_uri)
if ws_match:
    tenant_name = ws_match.groupdict().get("tenantname")
    ws_res_id = ws_match.groupdict().get("res_id")

print(tenant_name, ws_res_id)
from msrestazure.tools import parse_resource_id, resource_id

# parse resource id into components
ws_props = parse_resource_id(ws_res_id)
ws_props

microsoft.onmicrosoft.com /subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourceGroups/soc/providers/Microsoft.OperationalInsights/workspaces/cybersecuritysoc/Overview


{'subscription': 'd1d8779d-38d7-4f06-91db-9cbc8de0176f',
 'resource_group': 'soc',
 'namespace': 'Microsoft.OperationalInsights',
 'type': 'workspaces',
 'name': 'cybersecuritysoc',
 'children': '/Overview',
 'resource_parent': '',
 'resource_namespace': 'Microsoft.OperationalInsights',
 'resource_type': 'workspaces',
 'resource_name': 'cybersecuritysoc'}

In [100]:
from msrestazure.tools import parse_resource_id, resource_id, is_valid_resource_id
# Can we convert back and is resource ID valid?
print(resource_id(**ws_props))
is_valid_resource_id(resource_id(**ws_props))

True

## Queries to get workspace details from Resource graph


In [102]:
import msticpy

msticpy.init_notebook(verbosity=0)
az_qry = QueryProvider("ResourceGraph")
az_qry.connect()

True

In [99]:
ws_details_by_id = """
Resources
| where type == "microsoft.operationalinsights/workspaces"
| where id =~ "{resource_id}"
| extend workspaceId = tostring(properties["customerId"])
| project-rename workspaceName = name
| project workspaceName, workspaceId, tenantId, subscriptionId, resourceGroup, id
"""
ws_resource_details = (
    az_qry.exec_query(ws_details_by_id.format(resource_id=resource_id(**ws_props)))
    .iloc[0]
    .to_dict()
)
ws_resource_details

{'workspaceName': 'CyberSecuritySOC',
 'workspaceId': '8ecf8077-cf51-4820-aadd-14040956f35d',
 'tenantId': '4b2462a4-bbee-495a-a0e1-f23ae524cc9c',
 'subscriptionId': 'd1d8779d-38d7-4f06-91db-9cbc8de0176f',
 'resourceGroup': 'soc',
 'id': '/subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourceGroups/soc/providers/Microsoft.OperationalInsights/workspaces/cybersecuritysoc'}

In [97]:
ws_details_by_name = """
Resources
| where type == "microsoft.operationalinsights/workspaces"
| where name =~ "{resource_name}"
  and ("{resource_group}" == "" or resourceGroup =~ "{resource_group}")
  and ("{subscription_id}" == "" or subscriptionId =~ "{subscription_id}")
| extend workspaceId = tostring(properties["customerId"])
| project-rename workspaceName = name
| project workspaceName, workspaceId, tenantId, subscriptionId, resourceGroup, id
"""
ws_resource_details = (
    az_qry.exec_query(
        ws_details_by_name.format(
            resource_name=ws_props["resource_name"],
            resource_group=ws_props["resource_group"],
            subscription_id="",
        )
    )
    .iloc[0]
    .to_dict()
)
# Note this query can return multiple rows - should check for that
ws_resource_details

{'workspaceName': 'CyberSecuritySOC',
 'workspaceId': '8ecf8077-cf51-4820-aadd-14040956f35d',
 'tenantId': '4b2462a4-bbee-495a-a0e1-f23ae524cc9c',
 'subscriptionId': 'd1d8779d-38d7-4f06-91db-9cbc8de0176f',
 'resourceGroup': 'soc',
 'id': '/subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourceGroups/soc/providers/Microsoft.OperationalInsights/workspaces/cybersecuritysoc'}

In [ ]:
ws_details_qry = """
Resources
| where type == "microsoft.operationalinsights/workspaces"
| extend workspaceId = tostring(properties["customerId"])
| where workspaceId =~ "{workspace_id}"
| project-rename workspaceName = name
| project workspaceName, workspaceId, tenantId, subscriptionId, resourceGroup, id
"""
ws_resource_details = (
    az_qry.exec_query(
        ws_details_qry.format(
            resource_name=ws_props["resource_name"],
            resource_group=ws_props["resource_group"],
        )
    )
    .iloc[0]
    .to_dict()
)
ws_resource_details

{'workspaceName': 'CyberSecuritySOC',
 'workspaceId': '8ecf8077-cf51-4820-aadd-14040956f35d',
 'tenantId': '4b2462a4-bbee-495a-a0e1-f23ae524cc9c',
 'subscriptionId': 'd1d8779d-38d7-4f06-91db-9cbc8de0176f',
 'resourceGroup': 'soc'}

# Get the tenant ID of the user (not necessarily the workspace)

In [88]:
tenant_uri = (
    "https://login.microsoftonline.com/{tenant_name}/.well-known/openid-configuration"
)

t_resp = requests.get(tenant_uri.format(tenant_name=tenant_name))
tenant_details = t_resp.json()

tenant_ep_rgx = r"(?P<endpoint>https://[^/]+)/(?P<tenant_id>[^/]+).*"
tenant_match = re.search(
    tenant_ep_rgx, tenant_details.get("token_endpoint")
).groupdict()
tenant_match

{'endpoint': 'https://login.microsoftonline.com',
 'tenant_id': '72f988bf-86f1-41af-91ab-2d7cd011db47'}

## Get the cloud for the tenant endpoint

In [95]:
from msrestazure import azure_cloud
from msticpy.auth.cloud_mappings import create_cloud_ep_dict

azure_cloud.AZURE_PUBLIC_CLOUD.endpoints

tenant_cloud = next(
    iter(
        cloud
        for cloud, aad in create_cloud_ep_dict("active_directory").items()
        if aad.casefold() == tenant_match["endpoint"].casefold()
    )
)


# Build settings for workspace and Azure

We'll use these to populate the config

In [ ]:
ws_settings = {
    "AzureSentinel": {
        "Workspaces": {
            ws_resource_details['workspaceName']: {
                "WorkspaceId": ws_resource_details['workspaceId'],
                "TenantId": tenant_match["tenant_id"],
                "SubscriptionId": ws_props["subscription"],
                "ResourceGroup": ws_props["resource_group"],
                "WorkspaceName": ws_resource_details['workspaceName']
            }
        }
    }
}

azure_settings = {
    "Azure": {
        "cloud": tenant_cloud,
        "auth_methods": ["cli", "msi"]
    }
}